# Twitter API with Python Example
##### Author: Jared Wilber


Use Python for:

- working with an API
- working with a database


In [4]:
# import what we need
from textblob import TextBlob
import tweepy
import dataset
from sqlalchemy.exc import ProgrammingError
import json

## API Authentication

Sign up here to get credentials:
https://apps.twitter.com/

In [2]:
# Define constants
TWITTER_APP_KEY = "zS2Ex9moP635A6Nu5G6uJfaPu"
TWITTER_APP_SECRET = "towF0X2m59jtFRwudf2X1edKbpTEXXneZD6tDLo0FDiGJCxXFH"
TWITTER_KEY = "796875093456494592-XTA7Khp5LZVErrvUE30fkhnGAPhuQk3"
TWITTER_SECRET = "kArRkH2zld4b68qtMyr9KzRb5alyRCyLPsUpAGwYDZX4w"

TRACK_TERMS = ['Kuffar', 'beheading', 'kufr', 'isis', 'daesh', 'deathtoamerica', 'ihateamerica']


## Stream Tweets into db

We'll use sqlite. It can hold 140 terrabytes == solid for any proof of concept

In [3]:
# Set up data base stuff
DB_STRING = "sqlite:///tweets.db"
TABLE_NAME = "tweets"
# Set up data base
db = dataset.connect(DB_STRING)

# csv name for later
CSV_NAME = "tweets.csv"

In [4]:
# Define StreamListener Class
class StreamListener(tweepy.StreamListener):

    def on_status(self, status):
        
        # drop tweets tha are rts
        if hasattr (status, 'retweeted_status'):              
            return
        
        
        # Tweet details we want to keep
        description = status.user.description
        loc = status.user.location
        text = status.text
        print status.text, "\n"
        coords = status.coordinates
        name = status.user.screen_name
        user_created = status.user.created_at
        followers = status.user.followers_count
        id_str = status.id_str
        created = status.created_at
        retweets = status.retweet_count
        blob = TextBlob(text)
        sent = blob.sentiment

        if coords is not None:
            coords = json.dumps(coords)
        
        # Stream insto database
        table = db[TABLE_NAME]
        try:
            table.insert(dict(
                user_description=description,
                user_location=loc,
                coordinates=coords,
                text=text,
                user_name=name,
                user_created=user_created,
                user_followers=followers,
                id_str=id_str,
                created=created,
                retweet_count=retweets,
                polarity=sent.polarity,
                subjectivity=sent.subjectivity,
            ))
            
        except ProgrammingError as err:
            print(err)

    def on_error(self, status_code):
        if status_code == 420:
            #returning False if on_data disconnects the stream
            return False

In [5]:
# Set up Streamer
auth = tweepy.OAuthHandler(TWITTER_APP_KEY, TWITTER_APP_SECRET)
auth.set_access_token(TWITTER_KEY,TWITTER_SECRET)
api = tweepy.API(auth)

In [6]:
# Run it
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
# lesgetit
stream.filter(track=TRACK_TERMS)

Links verliest de pedalen, aflevering 24.985.356

 https://t.co/OvUxMYvstx 

It's thoughtless delusional fear-mongering acts as yours Bibi .@Netanya &amp;Nazis as #Sessions Trump&amp;Saudi'+Irans's ISIS setting #Earth ablaze 

Not letting Muslims run your cultural institutions will make those fine people transform into terrorists. Got it. https://t.co/eaBsGt58WH 

ISIS fighters try hiding among Mosul families as refugee rates surge - Fox News https://t.co/PkHEYiGeJ2 

Findings by ICCT of #Daesh suicide attacks- 83% against military targets; 80% by local recruits : https://t.co/wGHc6cK6e4 via @ICCT_TheHague 

Riz Ahmed warns lack of diversity on TV will drive young to Isis https://t.co/zbmzwDE00e 

@lisa_w227  Bill won and we still had deregulation, welfare reform to states, tech crash, war in iraq, housing crash and then ISIS. 

#Raqqa photos for the ruins Due Airstrikes on #Taqbah city #Syria #ISIS https://t.co/SDwnKtSWDm 

@TheRealDevonA @SenWarren Obama/Hillary armed ISIS. Putin sta

KeyboardInterrupt: 

# Now we've successfully scraped tweets, but how can we use them?

## Step 1: Extract tweets from db and put them into a useable file (we'll do .csv)

In [7]:
!ls
# no tweets.csv file

#Import the necessary methods from tweepy library.py
BeautifulSoup.ipynb
Big_Data_Python.ipynb
MoMAExhibitions1929to1989.csv
Tweepy_Notebook.ipynb
Untitled.ipynb
Untitled1.ipynb
academy_awards.csv
convert_db_to_csv.py
scrape.py
settings.py
settings.pyc
tweets.db


### Make csv file from db

In [30]:
db = dataset.connect(DB_STRING)

# save db data to 'result'
result = db[TABLE_NAME].all()
# coerce to csv
dataset.freeze(result, format='csv', filename=CSV_NAME)

In [31]:
!ls
# dope -  there's a csv file now

#Import the necessary methods from tweepy library.py
BeautifulSoup.ipynb
Big_Data_Python.ipynb
MoMAExhibitions1929to1989.csv
Tweepy_Notebook.ipynb
Untitled.ipynb
academy_awards.csv
convert_db_to_csv.py
scrape.py
settings.py
settings.pyc
tweets.csv
tweets.db


### View Data

In [34]:
import pandas as pd
tweets = pd.read_csv("tweets.csv")

In [35]:
tweets.head()

,id,polarity,user_followers,text,user_description,created,user_location,coordinates,id_str,subjectivity,user_created,retweet_count,user_name
0,1,0.5,642,"@ScottJonesy Yes, they are.\n\nAnd now that Pr...",65 year old 'local haole' Surf Bum & Visionary...,2017-02-27T23:32:41,Kailua Hawaii,NaN,836358421272145920,0.500000,2012-03-18T22:16:19,0,HURRICANEPAUL
1,2,0.1,2507,Omar Matteen like Timothy McVeigh American Cit...,Covering College Football Coaching from Miami ...,2017-02-27T23:32:42,Atlanta - Boulder - Tahoe - SF,NaN,836358426288611328,0.166667,2009-06-06T01:54:44,0,CoachesHotSeat
2,3,0.0,43,"@Newsweek ISIS is an American creation, it's j...","French book/film writer (Cheneyland, Charlie's...",2017-02-27T23:32:54,London,NaN,836358475886260225,0.000000,2011-09-05T12:44:03,0,FelsonJack
3,4,0.0,53203,"Obama, Co-founder of #ISIS, Lied Abt Intellige...","Free lance writer, to right of Attila, #Cathol...",2017-02-27T23:32:55,People's Republic of Chicago,NaN,836358483200991232,0.000000,2009-12-12T23:38:13,0,gerfingerpoken
4,5,0.0,164,😂😂😂 https://t.co/qMDmmiaKY3,Millennium varsity baseball #34// Joshua 1:9,2017-02-27T23:32:56,NaN,NaN,836358485071667204,0.000000,2015-12-13T01:11:14,0,Atizz__14


### Get sentiment for tweets

In [36]:
from textblob import TextBlob

def get_sentiment(text):
    
    text = text.decode("utf8") # decode if unreadable ascii
    blob = TextBlob(text)
    sent = blob.sentiment.polarity
    return sent

tweets['sentiment'] = tweets['text'].apply(lambda x: get_sentiment(x))

In [37]:
tweets.head(n=15)

,id,polarity,user_followers,text,user_description,created,user_location,coordinates,id_str,subjectivity,user_created,retweet_count,user_name,sentiment
0,1,0.500000,642,"@ScottJonesy Yes, they are.\n\nAnd now that Pr...",65 year old 'local haole' Surf Bum & Visionary...,2017-02-27T23:32:41,Kailua Hawaii,NaN,836358421272145920,0.500000,2012-03-18T22:16:19,0,HURRICANEPAUL,0.500000
1,2,0.100000,2507,Omar Matteen like Timothy McVeigh American Cit...,Covering College Football Coaching from Miami ...,2017-02-27T23:32:42,Atlanta - Boulder - Tahoe - SF,NaN,836358426288611328,0.166667,2009-06-06T01:54:44,0,CoachesHotSeat,0.100000
2,3,0.000000,43,"@Newsweek ISIS is an American creation, it's j...","French book/film writer (Cheneyland, Charlie's...",2017-02-27T23:32:54,London,NaN,836358475886260225,0.000000,2011-09-05T12:44:03,0,FelsonJack,0.000000
3,4,0.000000,53203,"Obama, Co-founder of #ISIS, Lied Abt Intellige...","Free lance writer, to right of Attila, #Cathol...",2017-02-27T23:32:55,People's Republic of Chicago,NaN,836358483200991232,0.000000,2009-12-12T23:38:13,0,gerfingerpoken,0.000000
4,5,0.000000,164,😂😂😂 https://t.co/qMDmmiaKY3,Millennium varsity baseball #34// Joshua 1:9,2017-02-27T23:32:56,NaN,NaN,836358485071667204,0.000000,2015-12-13T01:11:14,0,Atizz__14,0.000000
5,6,0.000000,19,@DCorleone62 Abu Ghraib was the cause of ISIS.,"Yeah, they are nice. Thanks!",2017-02-27T23:32:59,NaN,NaN,836358497323347968,0.000000,2013-02-25T00:35:05,0,ShellyElter,0.000000
6,7,0.000000,32,Must read this piece by @kurteichenwald . Ign...,I will not be ruled by a six year old prone to...,2017-02-27T23:33:01,NaN,NaN,836358508421459968,0.000000,2014-01-24T14:26:24,0,MikeRose2d,0.000000
7,8,0.000000,27950,Sailor beheaded by #ISIS #Terror on video http...,Religion of Peace or Agenda Of Evil What is Is...,2017-02-27T23:33:07,Mecca,NaN,836358530479218688,0.000000,2016-01-11T17:47:07,0,AgendaOfEvil,0.000000
8,9,-0.155556,296,Moscow Means Business: Russia Forms Elite Syri...,"The Truth Hurts! PC = Mind Control, Over-Popul...",2017-02-27T23:33:07,NaN,NaN,836358531645341697,0.288889,2015-05-06T01:45:44,0,pcliers,-0.155556
9,10,0.000000,94,Share if u think #ISIS should be wiped off the...,MFI international Pakistan (I'm Anti #ISIS #IS...,2017-02-27T23:33:07,NaN,NaN,836358531523702784,0.000000,2014-10-18T17:22:10,0,AmjadHussain299,0.000000


## More: Using SQL through Python

In [38]:
import sqlite3
conn = sqlite3.connect("tweets.db")

In [19]:
# write SQL query
query = "SELECT text, user_name FROM tweets WHERE polarity < 0.0 LIMIT 10;"

In [21]:
result = conn.execute(query).fetchall()

result

[(u'Moscow Means Business: Russia Forms Elite Syrian Army Unit to Hunt Down ISIS: https://t.co/a91WPzXDEK',
  u'pcliers'),
 (u'TV journalist Shifa Gardi killed in Iraq covering battle to reclaim Mosul from ISIS: https://t.co/NJRi0M3EQZ https://t.co/xbTNpCdpBV',
  u'Findsuitablejob'),
 (u'If you can watch this and not want our borders locked down https://t.co/MFzNrj5pDD are not human these are animals https://t.co/ZgwOnGUg5X',
  u'ohmy58')]

### Read sql query to pandas

In [22]:
query_result = pd.read_sql_query(query, conn)
query_result

,text,user_name
0,Moscow Means Business: Russia Forms Elite Syri...,pcliers
1,TV journalist Shifa Gardi killed in Iraq cover...,Findsuitablejob
2,If you can watch this and not want our borders...,ohmy58


# Load your own csv into db

data link: https://www.aggdata.com/awards/oscar
   

In [ ]:
df = pd.read_csv("academy_awards.csv")
df

In [ ]:
# get year only as 64bit int
df["Year"] = df["Year"].str[0:4]
df["Year"] = df["Year"].astype("int64")

In [ ]:
print df.shape
df.head(n=4)

In [ ]:
# Get only award categories we want in 'recent' years

# 1st: Subset dataframe so we only have recent years
later_than_2000 = df[df["Year"] > 2000]

# 2nd: Use subsetted data frame to extract recent years
award_categories = ["Actor -- Leading Role","Actor -- Supporting Role", "Actress -- Leading Role", "Actress -- Supporting Role"]
nominations = later_than_2000[later_than_2000["Category"].isin(award_categories)]

In [ ]:
nominations.shape
nominations.head(n=5)

In [ ]:
# Binarize Won columns
replacements = { "NO": 0, "YES": 1 }
nominations["Won?"] = nominations["Won?"].map(replacements)
nominations["Won"] = nominations["Won?"]
drop_cols = ["Won?","Unnamed: 5", "Unnamed: 6","Unnamed: 7", "Unnamed: 8", "Unnamed: 9", "Unnamed: 10"]
final_nominations = nominations.drop(drop_cols, axis=1)

In [ ]:
final_nominations

In [ ]:
# Clean final columns
additional_info_one = final_nominations["Additional Info"].str.rstrip("'}")
additional_info_two = additional_info_one.str.split(" {'")
movie_names = additional_info_two.str[0]
characters = additional_info_two.str[1]
final_nominations["Movie"] = movie_names
final_nominations["Character"] = characters
final_nominations = final_nominations.drop("Additional Info", axis=1)

In [ ]:
final_nominations

## Export newly cleaned csv to sql

In [ ]:
import sqlite3
conn = sqlite3.connect("nominations.db")
conn.text_factory=str
final_nominations.to_sql("nominations", conn, index=False)

In [ ]:
query_one = "pragma table_info(nominations);"
print conn.execute(query_one).fetchall()

In [ ]:
query_two = "select * from nominations limit 10;"
print conn.execute(query_two).fetchall()
conn.close()